In [ ]:
__nbid__ = '0071'
__author__ = 'Brian Merino <brian.merino@noirlab.edu>, Vinicius Placco <vinicius.placco@noirlab.edu>'
__version__ = '20250515' # yyyymmdd; Versión DataSpamp de este notebook
__keywords__ = ['GHOST','Gemini','stars','DRAGONS']

# Gemini Ghost xx Reducción de Oph usando Dragons Python API
***
## Datos de archivo público de Ghost_tutorial-GS-ENG-GHOST-COM-3-915 (XX OPH)
### adaptado de https://dragons.readthedocs.io/projects/ghost-drtutorial/en/stable/
***

### Nota: Este notebook puede tardar más de una hora en ejecutarse. Puede tomar más o menos tiempo si trabaja con un conjunto de datos diferente del utilizado en este tutorial. El tiempo de ejecución total depende del número y la variedad de archivos que desea reducir.

### Si desea ejecutar este notebook en su máquina local, asegúrese de que la base de datos de calibración de los dragones esté configurada correctamente. Para hacerlo, asegúrese de que la sección de calibraciones de ./dragons/dragonsrc se vea así:

`` `` ``
[Calibs]
bases de datos = ~/.dragons/dragons.db get store
`` `` ``

## Tabla de contenido
* [Objetivos](#Objetivos)
* [Resumen](#Resumen)
* [Descargos y atribución](#Descargo de responsabilidad)
* [Importaciones y configuración](#importaciones)
* [Sobre el conjunto de datos](#sobre)
* [Prepare el directorio de trabajo](#Preparar)
* [Descarga de datos para reducción](#descarga_data)
* [Crear listas de archivos](#file_lists)
* [Configurar el Logger Dragons](#dragons_logger)
* [Crear update_list () y reduce_func ()](#func)
* [Reducción de sesgo](#sesgos)
* [Reducción plana](#Flats)
* [Reducción del arco](#arcs)
* [Reducción estándar espectroscópica](#spec_standard)
* [Reducción de la ciencia](#Science)
* [Gráfica de espectros reducidos](#gráfica)
* [Salida 1D Spectra](#Write1DSPectra)
* [Guardar gráficos de espectros reducidos](#save_plot)
* [Haga compatible con espectros reducidos IRAF](#IRAF_Compatible)

<a class="anchor" id="attribution"></a>
# Avisos Legales y atribuciones

Avisos Legales
-----------
Tome en cuenta que usar el Astro Data Lab constituye un acuerdo con nuestros [Avisos Legales](https://datalab.noirlab.edu/disclaimers.php) mínimos.

Reconocimientos
---------------
Si ud. usa el **Astro Data Lab** en sus publicaciones de investigación, por favor incluya el siguiente texto en la sección de Reconocimientos de su publicaciones:

_Esta investigación utiliza servicios de datos proveeidos por el Astro Data Lab, el cual es parte del Programa "Community Science and Data Center" (CSDC) (Centro de Ciencia Comunitaria y Datos) del NSF NOIRLab. NOIRLab es operado por la "Association of Universities for Research in Astronomy (AURA), Inc."(Asociación de Universidaddes para la Investigación en Astronomía, Inc.), bajo un acuerdo de cooperación con la "U.S. National Science Foundation" (Fundación Nacional de Ciencia de los EE. UU.)._

Si utiliza **SPARCL junto con la plataforma de Astro Data Lab** (por medio de JupyterLab, línea de comando o interfaz de la web) en su publicación de investigación, por favor incluya el siguiente texto en la sección de Reconocimientos de su publicaciones:

_Esta investigación utiliza servicios o datos proporcionados por el "SPectra Analysis and Retrievable Catalog Lab" (SPARCL) (Laboratorio de Análisis y Catálogo Recuperable de Espectros) y el Astro Data Lab, ambos pertenecientes al Programa "Community Science and Data Center" (CSDC) (Centro de Ciencia Comunitaria y Datos) de NSF NOIRLab. NOIRLab es operado por la "Association of Universities for Research in Astronomy (AURA), Inc." (Asociación de Universidades para la Investigación en Astronomía, Inc.), bajo un acuerdo de cooperación con la "U.S. National Science Foundation" (Fundación Nacional de Ciencia de los EE. UU.)._

En cualquiera de los casos, **por favor cite las siguientes publicaciones**:

* Publicación del concepto de Data Lab: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, https://doi.org/10.1117/12.2057445

* Descripción general del Astro Data Lab: Nikutta et al., "Data Lab - A Community Science Platform", Astronomy and Computing, 33, 2020, https://doi.org/10.1016/j.ascom.2020.100411.

Si hace referencia al Jupyterlab / Jupyter notebooks de Data Lab, cite:

* Juneau et al., "Jupyter-Enabled Astrophysical Analysis Using Data-Proximate Computing Platforms", CiSE, 23, 15, 2021, https://doi.org/10.1109/MCSE.2021.3057097.

Si publica en una revista de la AAS, agregue también la palabra clave `\facility{Astro Data Lab}`

Y si está usando SPARCL, por vor agregue también `\software{SPARCL}` y cite:

* Juneau et al., "SPARCL: SPectra Analysis and Retrievable Catalog Lab", Conference Proceedings for ADASS XXXIII, 2024
https://doi.org/10.48550/arXiv.2401.05576.

La biblioteca de NOIRLab mantiene [listas de reconocimientos apropiados](https://noirlab.edu/science/about/scientific-acknowledgments) para usar cuando se hacen publicaciones utilizando los recursos, servicios o datos del Laboratorio.

---- **Versión en Inglés** ----


# Disclaimer & attribution

Disclaimers
-----------
Note that using the Astro Data Lab constitutes your agreement with our minimal [Disclaimers](https://datalab.noirlab.edu/disclaimers.php).

Acknowledgments
---------------
If you use **Astro Data Lab** in your published research, please include the text in your paper's Acknowledgments section:

_This research uses services or data provided by the Astro Data Lab, which is part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

If you use **SPARCL jointly with the Astro Data Lab platform** (via JupyterLab, command-line, or web interface) in your published research, please include this text below in your paper's Acknowledgments section:

_This research uses services or data provided by the SPectra Analysis and Retrievable Catalog Lab (SPARCL) and the Astro Data Lab, which are both part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

In either case **please cite the following papers**:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, https://doi.org/10.1117/12.2057445

* Astro Data Lab overview: Nikutta et al., "Data Lab - A Community Science Platform", Astronomy and Computing, 33, 2020, https://doi.org/10.1016/j.ascom.2020.100411

If you are referring to the Data Lab JupyterLab / Jupyter Notebooks, cite:

* Juneau et al., "Jupyter-Enabled Astrophysical Analysis Using Data-Proximate Computing Platforms", CiSE, 23, 15, 2021, https://doi.org/10.1109/MCSE.2021.3057097

If publishing in a AAS journal, also add the keyword: `\facility{Astro Data Lab}`

And if you are using SPARCL, please also add `\software{SPARCL}` and cite:

* Juneau et al., "SPARCL: SPectra Analysis and Retrievable Catalog Lab", Conference Proceedings for ADASS XXXIII, 2024
https://doi.org/10.48550/arXiv.2401.05576

The NOIRLab Library maintains [lists of proper acknowledgments](https://noirlab.edu/science/about/scientific-acknowledgments) to use when publishing papers using the Lab's facilities, data, or services.


<a class = "Anchor" id = "importaciones"> </a>
# Importaciones y configuración

In [ ]:
import warnings
import glob
import os
import numpy as np
import astrodata
import shutil
import matplotlib.pyplot as plt

from gempy.utils import logutils
from gempy.adlibrary import dataselect

from recipe_system import cal_service
from recipe_system.reduction.coreReduce import Reduce

from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)
warnings.filterwarnings("ignore")

## Establezca una función para verificar y eliminar varios directorios o archivos.

In [ ]:
def check_and_delete(path):
    # Verifique si la ruta ya existe.
    if os.path.exists(path):
        # Si la ruta conduce a un directorio.
        if os.path.isdir(path):
            # Eliminar directorio
            shutil.rmtree(path)
            
        # Si la ruta conduce a un archivo.
        elif os.path.isfile(path):
            # Eliminar archivo
            os.remove(path)

<a class = "Anchor" id = "Acerca"> </a>
# Sobre el conjunto de datos

Los datos de fantasmas utilizados para este tutorial son de Star XX Oph. IFU-1 se utilizó para obtener la resolución estándar de la estrella. Los datos se obtuvieron durante la ejecución de la puesta en marcha.

La siguiente tabla contiene un resumen del conjunto de datos:

| Tipo de observación | Nombre del archivo (s) | IFU, Binning y Leer Modo |
| : --- | : --- | : ---: |
| Ciencia | S20230416S0079 | Azul: 2x2, lento; Rojo: 2x2, medio |
| Sesgo de ciencias | S20230417S0011-015 |  |
| Pisos de ciencias | S20230416S0047 | 1x1; azul: lento; Rojo: Medio |
| Arcs de ciencias | S20230416S0049-51 | 1x1; azul: lento; Rojo: Medio |
| Sesgos planos <br> sesgos de arco | S20230417S0036-40 | 1x1; azul: lento; Rojo: Medio |
| Estándar (CD -32 9927) | S20230416S0073 | Azul: 2x2, lento; Rojo: 2x2, medio |
| Sesgos estándar <br> Pisos estándar <br> Arco estándar <br> Pargos planos estándar <br> Biste de arco estándar | Use calibraciones científicas |
| BPMS | BPM_20220601_GHOST_BLUE_11_FULL_4AMP.FITS <br> BPM_20220601_GHOST_RED_11_FULL_4AMP.FITS |  |

<a class = "Anchor" id = "Preparar"> </a>
# Prepare el directorio de trabajo

Si tiene algún archivo intermedio que se creó al ejecutar este código en el pasado, deberá eliminarlos de su directorio de trabajo. La siguiente celda define una función de limpieza que eliminará todos los archivos de ajuste de su directorio de trabajo. Esta función se volverá a llamar al final del tutorial, dejándote solo con el producto final. 

De manera predeterminada, esta función eliminará todos los archivos de registro, lista y ajuste en el directorio de trabajo. Si hay archivos que se han reducido previamente que le gustaría conservar, establezca `save_reduced = 1` al llamar a la función.

In [ ]:
def clean_up(save_reduced=0):
    # ¿El directorio de calibraciones ya existe?
    check_and_delete('calibrations')

    # Eliminar archivos de registro y lista existentes.
    work_dir_path = os.getcwd()
    work_dir = os.listdir(work_dir_path)

    for item in work_dir:
        if item.endswith(".log") or item.endswith(".list"):
            check_and_delete(os.path.join(work_dir_path,item))
    
    # A continuación, eliminaremos todos los archivos de ajuste existentes, excepto los archivos reducidos previamente, dependiendo de lo que establezca save_reduced.
    if save_reduced:
        all_files = glob.glob('*.fits')
        save = dataselect.select_data(all_files, [], ['PROCESSED'])
        
        for s in save:
            check_and_delete(os.path.join(work_dir_path,s))

        if os.path.exists(os.path.join('reduced')):
            remain_files = os.listdir(work_dir_path)
        
            for item in remain_files:
                if os.path.splitext(item)[1] in ('.dat','.pdf','.fits','.png'):
                    # Compruebe si el archivo ya existe en formato reducido/
                    # Si lo hace, elimínelo y reemplácelo con la nueva copia
                    if os.path.exists(os.path.join(work_dir_path,'reduced',item)):
                        check_and_delete(os.path.join(work_dir_path,'reduced',item))
                        shutil.move(item,os.path.join(work_dir_path,'reduced'))

                    else:
                        shutil.move(item,os.path.join(work_dir_path,'reduced'))
        
        # Crear directorio reducido/ y mover archivos reducidos
        else:
            os.mkdir(os.path.join(work_dir_path,'reduced'))
            remain_files = os.listdir(work_dir_path)
        
            for item in remain_files:
                if os.path.splitext(item)[1] in ('.dat','.pdf','.fits','.png'):
                    shutil.move(item,os.path.join(work_dir_path,'reduced'))

    else:
        all_files = glob.glob('*.fits')
        for a in all_files:
            check_and_delete(os.path.join(work_dir_path,a))

## Crear un directorio para archivos sin procesar
Este tutorial creará una gran cantidad de archivos intermedios que se almacenarán temporalmente en el directorio de trabajo. Para garantizar que no se pierda ninguno de los datos originales, crearemos un directorio llamado RAW para almacenar los datos preliminares de manera segura.

In [ ]:
check_and_delete('raw')
os.mkdir('raw')

<a class = "Anchor" id = "descargar_data"> </a>
# Descargar los datos

Descargue los datos espectroscópicos y de calibración del archivo Gemini al directorio de trabajo actual. Este paso solo necesita ser ejecutado una vez.

Si ejecuta este notebook por primera vez y necesita descargar el conjunto de datos, configure la variable "download = true". El notebook no se volverá a descargar el conjunto de datos si está configurado en falso. Esto se volverá particularmente útil si ejecuta los notebooks más de una vez.

In [ ]:
%%bash 

# Cree una lista de archivos FIT para descargar.
echo "\
https://archive.gemini.edu/file/S20230416S0047.fits
https://archive.gemini.edu/file/S20230416S0049.fits
https://archive.gemini.edu/file/S20230416S0050.fits
https://archive.gemini.edu/file/S20230416S0051.fits
https://archive.gemini.edu/file/S20230416S0073.fits
https://archive.gemini.edu/file/S20230416S0079.fits
https://archive.gemini.edu/file/S20230417S0011.fits
https://archive.gemini.edu/file/S20230417S0012.fits
https://archive.gemini.edu/file/S20230417S0013.fits
https://archive.gemini.edu/file/S20230417S0014.fits
https://archive.gemini.edu/file/S20230417S0015.fits
https://archive.gemini.edu/file/S20230417S0036.fits
https://archive.gemini.edu/file/S20230417S0037.fits
https://archive.gemini.edu/file/S20230417S0038.fits
https://archive.gemini.edu/file/S20230417S0039.fits
https://archive.gemini.edu/file/S20230417S0040.fits
https://archive.gemini.edu/file/bpm_20220601_ghost_blue_11_full_4amp.fits
https://archive.gemini.edu/file/bpm_20220601_ghost_red_11_full_4amp.fits\
" > ghost.list

In [ ]:
%%bash

download="True"

if [ $download == "True" ]; then
    wget --no-check-certificate -N -q -P './raw' -i ghost.list

else
    echo "Skipping download. To download the data set used in this notebook, set download=True."
fi

<a class = "FASCOR" ID = "FILE_LISTS"> </a>
# Crear listas de archivos

Este conjunto de datos contiene marcos de ciencia y calibración. Para algunos programas, podría tener diferentes objetivos observados y tiempos de exposición dependiendo de cómo organice sus datos sin procesar. La tubería de reducción de datos de Dragons no organiza los datos por usted. Tienes que hacerlo. Dragons proporciona herramientas para ayudarlo con eso.

El primer paso es crear listas que se utilizarán en el proceso de reducción de datos.

In [ ]:
all_files = glob.glob('raw/S2023*.fits')
all_files.append(glob.glob('raw/bpm*.fits')[0])
all_files.append(glob.glob('raw/bpm*.fits')[1])
all_files.sort()

<a class = "Anchor" id = "dragons_logger"> </a>
# Configuración del Logger Dragons

Dragons viene con un administrador de calibración local que utiliza las mismas reglas de asociación de calibración que el archivo del Observatorio Géminis. Esto permite reducir las solicitudes a una base de datos local de peso ligero para las calibraciones procesadas que coinciden cuando sea necesario para reducir un conjunto de datos.

Esto le dice al sistema dónde colocar la base de datos de calibración. Esta base de datos realizará un seguimiento de las calibraciones procesadas que le enviaremos.

In [ ]:
logutils.config(file_name='ghost.log')
caldb = cal_service.set_local_database()
caldb.init("w")

## Agregue las máscaras de píxel malas a la base de datos de calibración

In [ ]:
caldb.add_cal(glob.glob('raw/bpm*.fits')[0])
caldb.add_cal(glob.glob('raw/bpm*.fits')[1])

<a class = "Anchor" id = "func"> </a>
# update_list () y reduce_func ()

Este notebook requerirá actualizar la lista de archivos en su directorio de trabajo y llamar al comando reducir varias veces. Para reducir el texto repetitivo, hemos creado dos funciones que reducirán el número de líneas incluidas en este notebook.

In [ ]:
def update_list():
    # Cree una nueva lista de archivos que contenga los archivos intermedios
    # Identificar todos los archivos en el directorio de trabajo
    intermediate = os.listdir()
    new_all_files = []

    # Dado que OS.ListDir () devuelve todos los archivos en el directorio de trabajo
    # Este bucle elegirá solo los archivos ajustados y los agregará a una lista.
    for i in intermediate:
        if os.path.splitext(i)[1] == '.fits':
            new_all_files.append(i)
    
    print('%i archivos en la lista.'%len(new_all_files))
    return new_all_files

In [ ]:
def reduce_func(files_list,uparms=None,recipename=None):
    # Use la función de reducción de Dragons para reducir la lista proporcionada de archivos.
    # De manera predeterminada, esta función utilizará la configuración predeterminada para reducir ().
    # ARARMS: Esta es una lista de tuplas con el nombre primitivo y el parámetro en el primer elemento y el valor en el segundo p. Ej. [('StackFrames: Operation', 'Median')].
    # nombre de receta: el nombre de la receta para usar, p. 'MakeIRAFCompatible'.
    reduce = Reduce()
    reduce.files.extend(files_list)

    if uparms is not None:
        reduce.uparms = [uparms]
    
    if recipename is not None:
        reduce.recipename = recipename
    
    reduce.runr()

<a class = "Anchor" id = "Bierys"> </a>
# Reducción de sesgo

Comience utilizando DataSelect para identificar todos los archivos en All_Files con la etiqueta 'BIAS'.

In [ ]:
biasbundles = dataselect.select_data(all_files, ['BIAS'], [])
print(biasbundles)

## use reduce_func () para reducir los sesgos

Cuando se realice esta celda en ejecución, se crearán tres archivos para cada sesgo (ciencia, plano y arco). Tendrán el sufijo *_blue001.fits, *_red001.fits y *_slit.fits.

In [ ]:
reduce_func(biasbundles)

## Actualizar lista

La función reduce() de DRAGONS crea una gran cantidad de archivos intermedios que se almacenan en el directorio de trabajo. Antes de volver a llamarlo, primero debemos actualizar nuestra lista de archivos usando update_list().

In [ ]:
new_all_files = update_list()

## Ahora use DataSelect para elegir los sesgos de hendidura.

In [ ]:
biasslit = dataselect.select_data(new_all_files, ['BIAS','SLIT'])
print(biasslit)

## Reduce los sesgos de hendidura.

Cuando se haga en ejecución, se creará un nuevo archivo llamado S20230417S0011_SLIT_BIAS.FITS.

In [ ]:
reduce_func(biasslit)

## Actualice la lista de archivos.

In [ ]:
new_all_files2 = update_list()

## Use DataSelect para elegir los sesgos de la ciencia roja y reducirlos.

In [ ]:
expression = "binning==\'2x2\'"
parsed_expr = dataselect.expr_parser(expression)
biasredsci = dataselect.select_data(new_all_files2, ['BIAS', 'RED'], [], parsed_expr)

Una vez que termine de ejecutarse, existirá un nuevo archivo llamado S20230417S0011_red001_bias.fits en el directorio de trabajo.

In [ ]:
reduce_func(biasredsci)

## Seleccione los sesgos de ciencia azul y reduzca.

In [ ]:
expression = "binning=='2x2'"
parsed_expr = dataselect.expr_parser(expression)
biasbluesci = dataselect.select_data(new_all_files2, ['BIAS','BLUE'], [], parsed_expr)

Se creará un solo archivo llamado S20230417S0011_Blue001_bias.fits después de ejecutar esta celda.

In [ ]:
reduce_func(biasbluesci)

## Seleccione los sesgos rojos planos/arco y reduzca.

In [ ]:
expression = "binning=='1x1'"
parsed_expr = dataselect.expr_parser(expression)
biasredflatarc = dataselect.select_data(new_all_files2, ['BIAS','RED'], [], parsed_expr)

Ejecutar la siguiente celda creará un nuevo archivo llamado S20230417S0036_RED001_BIAS.FITS.

In [ ]:
reduce_func(biasredflatarc)

## Seleccione los sesgos azules/arcos planos y reduzca.

In [ ]:
expression = "binning=='1x1'"
parsed_expr = dataselect.expr_parser(expression)
biasblueflatarc = dataselect.select_data(new_all_files2, ['BIAS','BLUE'], [], parsed_expr)

Ejecutar la siguiente celda creará un nuevo archivo llamado S20230417S0036_BLUE001_BIAS.FITS.

In [ ]:
reduce_func(biasblueflatarc)

## Limpieza

La reducción GHOST crea muchos archivos, a menudo grandes, en el directorio de trabajo. Se recomienda realizar una limpieza entre cada fase de reducción. Si desea guardar los archivos intermedios, muévalos (mv) a otro lugar. En este tutorial, simplemente los eliminaremos.

In [ ]:
%%bash

rm *fits

<a class = "Anchor" id = "Flats"> </a>
# Reducción plana
## desacreditar los pisos, luego reducirlos. Comenzando con los pisos maestros y las hendiduras.

In [ ]:
flatbundles = dataselect.select_data(all_files, ['FLAT'], [])

Ejecutar esta celda generará 11 archivos del Cience Flat S20230416S0047.FITS. Cinco tendrán el sufijo _blue00*.fits, cinco tendrán el sufijo _red00*.fits, y el resto tendrá el sufijo _slit.fits.

In [ ]:
reduce_func(flatbundles)

## Actualice la lista de archivos.

In [ ]:
new_all_files3 = update_list()

## Seleccione las placas de hendidura y reduzca.

In [ ]:
slitflat = dataselect.select_data(new_all_files3, ['SLITFLAT'], [])

La siguiente celda creará un solo archivo llamado S20230416S0047_SLIT_SLITFLAT.FITS.

Nota: Esta celda dará un error con respecto a las entradas que tienen diferentes números de extensiones de SCI. Este es un problema conocido con los dragones y se mostrará independientemente de los datos que proporcione. Este error se puede ignorar de forma segura.

In [ ]:
reduce_func(slitflat)

## Seleccione y reduzca los pisos rojos.

In [ ]:
flatred = dataselect.select_data(new_all_files3, ['FLAT','RED'], [])

Ejecutar la siguiente celda también creará un solo archivo llamado S20230416S0047_RED001_FLAT.FITS.

In [ ]:
reduce_func(flatred)

## Seleccione y reduzca los pisos azules.

In [ ]:
flatblue = dataselect.select_data(new_all_files3, ['FLAT','BLUE'], [])

La siguiente celda creará un archivo llamado S20230416S0047_Blue001_flat.fits.

In [ ]:
reduce_func(flatblue)

## Limpieza

In [ ]:
%%bash

rm *fits

<a class = "Anchor" id = "Arcs"> </n>
# Arcos
## desacreditar los arcos y reducirlos.

In [ ]:
arcbundles = dataselect.select_data(all_files, ['ARC'], [])

Al ejecutar la siguiente celda se crearán 9 archivos, tres para cada uno de los arcos científicos. Tres tendrán el sufijo _blue001.fits, tres tendrán el sufijo _red001.fits y los tres restantes tendrán el sufijo _slit.fits.

In [ ]:
reduce_func(arcbundles)

## Actualice la lista de archivos.

In [ ]:
new_all_files4 = update_list()

## Seleccione y reduzca las imágenes de hendidura para el arco.

In [ ]:
arcslit_1 = dataselect.select_data(new_all_files4, ['ARC','SLIT'], [])

In [ ]:
# El tutorial original ordena sus listas numéricamente de forma predeterminada, mientras que esta versión no.
# Aquí se han agregado algunas líneas de código para ordenar manualmente la lista.
arcslit_1.sort()
arcslit_2 = [arcslit_1[0]]
print(arcslit_2)

La siguiente celda devolverá un solo archivo llamado S20230416S0049_SLIT_SLIT.FITS.

In [ ]:
reduce_func(arcslit_2)

## Seleccione y reduzca los arcos rojos.

In [ ]:
arcred = dataselect.select_data(new_all_files4, ['ARC','RED'], [])
arcred.sort()
arcred

La siguiente celda creará un archivo llamado S20230416S0049_RED001_ARC.FITS.

In [ ]:
reduce_func(arcred)

## Seleccione y reduzca los arcos azules.

In [ ]:
arcblue = dataselect.select_data(new_all_files4, ['ARC','BLUE'], [])
arcblue.sort()
arcblue

La siguiente celda también creará un solo archivo llamado S20230416S0049_BLUE001_ARC.FITS.

In [ ]:
reduce_func(arcblue)

## Limpieza

In [ ]:
%%bash

rm *fits

<a class = "Anchor" id = "Spec_Standard"> </a>
# Estándar espectroscópico
## desacreditar los estándares y reducirlos.

In [ ]:
expression = "object=='CD -32 9927'"
parsed_expr = dataselect.expr_parser(expression)
stdbundles = dataselect.select_data(all_files, [], [], parsed_expr)
stdbundles.sort()
stdbundles

La siguiente celda creará 5 archivos que comienzan con el mismo nombre que el estándar. Uno tendrá el sufijo _blue001.fits, tres tendrán el sufijo _red00*.fits y el archivo final tendrá el sufijo _slit.fits.

In [ ]:
reduce_func(stdbundles)

## Actualice la lista de archivos.

In [ ]:
new_all_files5 = update_list()

## Seleccione los datos estándar del visor de hendidura y redúzcalos.

In [ ]:
stdslit = dataselect.select_data(new_all_files5, ['SLIT'], [])
stdslit.sort()
stdslit

A diferencia de las celdas anteriores, la siguiente celda creará cuatro archivos de ajuste nuevos y un PDF. El PDF tendrá el sufijo _slit_slitflux. Un archivo ajustado tendrá el sufijo _slit_blue001_slit.fits y los tres restantes tendrán el sufijo _slit_red00*_slit.fits.

Nota: Esta celda dará un error con respecto a las entradas que tienen diferentes números de extensiones de SCI. Este es un problema conocido con los dragones y se mostrará independientemente de los datos que proporcione. Este error se puede ignorar de forma segura.

In [ ]:
reduce_func(stdslit)

## Seleccione los datos de estrella estándar rojo y reduzca.

In [ ]:
stdred = dataselect.select_data(new_all_files5, ['RED'], [])
stdred.sort()
stdred

Ejecución de la siguiente celda generará un archivo llamado S20230416S0073_RED001_STANDARD.FITS.

In [ ]:
reduce_func(stdred,uparms=('scaleCountsToReference:tolerance',1),recipename='reduceStandard')

## Seleccione los datos de estrella estándar azul y reduzca.

In [ ]:
stdblue = dataselect.select_data(new_all_files5, ['BLUE'], [])

In [ ]:
stdblue.sort()
stdblue

La siguiente celda también producirá un solo archivo llamado S20230416S0073_BLUE001_STANDARD.FITS.

In [ ]:
reduce_func(stdblue,uparms=('scaleCountsToReference:tolerance',1),recipename='reduceStandard')

<a class = "Anchor" id = "Clean_UP_3"> </a>
# Limpieza

In [ ]:
%%bash

rm *fits

<a class = "Anchor" id = "Science"> </a>
# Ciencia 
## Desmede los marcos científicos y reduzca.

In [ ]:
expression = "object=='XX Oph'"
parsed_expr = dataselect.expr_parser(expression)
scibundles = dataselect.select_data(all_files, [], [],parsed_expr)

Ejecutar la siguiente celda creará 5 archivos cuyos nombres comenzarán con el nombre del archivo de ciencias. Uno tendrá el sufijo _blue001.fits, tres tendrán el sufijo _red00*.fits y el archivo final tendrá el sufijo _slit.fits.

In [ ]:
reduce_func(scibundles)

## Actualice la lista de archivos.

In [ ]:
new_all_files6 = update_list()

<a class = "Anchor" id = "Science_Slit_Viewer"> </a>
# Seleccione los datos científicos de Slit-Viewer y reduzca.

In [ ]:
scislit  = dataselect.select_data(new_all_files6, ['SLIT'], [])

Al igual que cuando se reducen los datos científicos, la siguiente celda creará 5 archivos. Uno tendrá el sufijo _slit_blue001_slit.fits, tres tendrán el sufijo _slit_red00*_slit.fits y el último tendrá el sufijo _slit_slitflux.pdf.

Nota: Esta celda dará un ERROR con respecto a las entradas que tienen diferentes números de extensiones SCI. Este es un problema conocido con DRAGONS y aparecerá independientemente de los datos que proporciones. Este ERROR se puede ignorar con seguridad.

In [ ]:
reduce_func(scislit)

## Actualice la lista de archivos.

In [ ]:
new_all_files7 = update_list()

## Seleccione y reduzca los cuadros de la ciencia roja.

In [ ]:
expression = "object=='XX Oph'"
parsed_expr = dataselect.expr_parser(expression)
scired = dataselect.select_data(new_all_files7, ['RED'], [])
scired = np.sort(scired)

Ejecutar la siguiente celda creará 4 archivos. Uno tendrá el sufijo _red001_dragons.fits, y los otros tres tendrán el sufijo _red00*_calibrated.fits.

In [ ]:
reduce_func(scired)

## Seleccione y reduzca los marcos de ciencias azules.

In [ ]:
expression = "object=='XX Oph'"
parsed_expr = dataselect.expr_parser(expression)
sciblue = dataselect.select_data(new_all_files7, ['BLUE'], [])
sciblue = np.sort(sciblue)

Ejecutar la siguiente celda creará dos archivos. El primero tendrá el sufijo _blue001_calibrated.fits y el otro tendrá el sufijo _blue001_dragons.fits.

In [ ]:
reduce_func(sciblue)

<a class = "Anchor" id = "plot"> </a>
# Traza los espectros rojos y azules reducidos

In [ ]:
# Pantalla S20230416S0079_RED001_Dragons.fits y S20230416S0079_Blue001_dragons.fits
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(13,5))

red_file = 'S20230416S0079_red001_dragons.fits'
blue_file = 'S20230416S0079_blue001_dragons.fits'

red_pf  = astrodata.open(red_file)
blue_pf = astrodata.open(blue_file)

red_flux  = red_pf[0].data
blue_flux = blue_pf[0].data

red_flux_array  = np.array(red_flux)
blue_flux_array = np.array(blue_flux)

red_wave  = red_pf[0].wcs(np.arange(red_flux.size)).astype(np.float32)
blue_wave = blue_pf[0].wcs(np.arange(blue_flux.size)).astype(np.float32)

# Convierta el λ de nm a Å
red_wave_array  = np.array(red_wave*10)
blue_wave_array = np.array(blue_wave*10)

ax1.plot(blue_wave_array,blue_flux_array,lw=0.4)
ax1.set_xlim(3450,5450)
ax1.set_ylim(-0.05*10**(-12),0.2*10**(-12))
ax1.set_xlabel('Longitud de onda [$ \ aa $]')
ax1.set_ylabel('Flux [$ w m^{-2} nm^{-1} $]')
ax1.set_title(blue_file,size=11,fontweight='atrevido')

ax2.plot(red_wave_array,red_flux_array,lw=0.4)
ax2.set_xlim(5100,10700)
ax2.set_ylim(-0.05*10**(-12),0.23*10**(-12))
ax2.set_xlabel('Longitud de onda [$\AA$]')
ax2.set_ylabel('Flux [$ w m^{-2} nm^{-1} $]')
ax2.set_title(red_file,size=11,fontweight='atrevido')

plt.show()

<a class = "Anchor" id = "Write1DSPectra"> </a>
# Guardar espectros 1D

Si desea guardar los espectros terminados como archivos de texto en lugar de archivos ajustados, use la receta Write1DSPectra como se demuestra en las siguientes dos celdas. 

Ejecutar la primera celda creará dos archivos. El primero se llamará S20230416S0079_RED001_Dragons_001.DAT y el segundo se llamará S20230416S0079_RED001_DRAGONS_002.DAT.

Ejecutar la segunda celda también creará dos archivos. El primero se llamará S20230416S0079_BLUE001_Dragons_001.dat y el segundo se llamará S20230416S0079_BLUE001_Dragons_002.dat.

In [ ]:
red_to_1d = ['S20230416S0079_red001_dragons.fits']
reduce_func(red_to_1d,recipename='write1DSpectra')

In [ ]:
blue_to_1d = ['S20230416S0079_blue001_dragons.fits']
reduce_func(blue_to_1d,recipename='write1DSpectra')

<a class="ancla" id="Save_plot"></a>
# Guardar espectros reducidos

Si desea guardar un gráfico de sello PNG de los espectros rojo y azul reducidos, ejecute la siguiente celda. También tiene la opción de guardar la imagen en un formato diferente, incluidos SVG, eps y PS, reemplazando 'PNG' en la penúltima línea con el formato deseado. Este código guardará los espectros rojo y azul por separado. Un archivo guardado se llamará S20230416S0079_blue001_dragons.png y el otro se llamará S20230416S0079_red001_dragons.png.

In [ ]:
# .- Autor: David Herrera - junio de 2024
# Cree una lista de todos los archivos de ajuste reducidos de dragones en el directorio actual
ls_fits = 'ls -1 *{blue,red}00?*_dragons.fits > dragons_fits.list'
os.system(ls_fits)
# Guardar la lista de dragones reduce los ajustes en una lista
fits_list = 'dragons_fits.list'

# Abra la lista de archivos de ajuste
with open (fits_list, "r") as files:
    fnames_list = [line.strip() for line in files.readlines()]

# Lea cada nombre de archivo adecuado
for fname in fnames_list:
    # Determine si es un espectro rojo o azul
    file = str(fname.strip())
    if '_red' in file:
        band = 'red' 
    else:
        band = 'blue'
    # Abra y lea los datos de cada archivo ajustado
    ad = astrodata.open(file)
    flux = ad[0].data
    lam = ad[0].wcs(np.arange(flux.size)).astype(np.float32)
     
    # Convierta el λ de nm a Å
    lambda_array = np.array(lam*10)
    flux_array = np.array(flux)
 
    # Definir rangos de lambda para cada panel dependiendo de la banda
    if band == 'red':
        lambda_ranges = [(5370, 6330), (6270, 7230), (7170, 8130), (8070, 9030), (8970, 9930)]
    else:
        lambda_ranges = [(3790, 4110), (4090, 4410), (4390, 4710), (4690, 5010), (4990, 5310)]
    # Crear una figura y un conjunto de 5 subtramas
    fig, axs = plt.subplots(len(lambda_ranges), 1, sharex=False, figsize=(10, 8))
 
    # Trazar datos en cada rango
    for i, (lam_min, lam_max) in enumerate(lambda_ranges):
        # Filtrar datos para el rango actual
        mask = (lambda_array >= lam_min) & (lambda_array <= lam_max)
        lambda_filtered = lambda_array[mask]
        flux_filtered = flux_array[mask]
     
        if len(lambda_filtered) > 0 and len(flux_filtered) > 0:
            # Trazar los datos
            axs[i].plot(lambda_filtered, flux_filtered, c=band, lw='.6')
            # Calcule la mediana de flujo del rango de flujo en el rango de corriente
            flux_median = np.median(flux_filtered)
            # Establezca los Limits X
            axs[i].set_xlim(lam_min, lam_max)
            # Establezca los límites y para este panel en particular
            ylim=(-0.25 * flux_median, 2.5 * flux_median)
            axs[i].set_ylim(ylim)
            axs[i].set
            # Ocultar valores de tic en y
            # AXS [i] .set_yticks ([])
            # Manejo de garrapatas
            axs[i].minorticks_on()
            if band == 'red':
                axs[i].set_xticks(np.arange(lam_min+30,lam_max+30,step=150))
                axs[i].set_xticks(np.arange(lam_min+80,lam_max,step=50), minor = True)
            axs[i].tick_params(axis = 'y', which='major', labelsize = 8)
        else:
            # Manejar el caso donde no hay puntos de datos en el rango
            axs[i].text(0.5, 0.5, 'No data in this range', transform=axs[i].transAxes,
                     ha='center', va='center', color=band)
 
        # Opcionalmente, configure Y-Label para cada panel
        # axs [i] .set_yticks ()
        # Solo la etiqueta y en el 3er panel
        if i == 2: axs[i].set_ylabel('Flux [$ w m^{-2} nm^{-1} $]')
 
    # Establecer la etiqueta del eje X para la gráfica inferior
    axs[-1].set_xlabel('λ (Å)')
    # Establecer título para toda la trama
    fig.suptitle(file)
    # Ajuste el diseño para eliminar los huecos entre las subtramas
    plt.tight_layout()
 
    # Mostrar la trama
    # plt.show ()
 
    # Guardar graficar en un archivo (puede ser un PNG, SVG, EPS, PS)
    fig.savefig(file.strip('fits') + 'png', dpi='figure', format='png', metadata=None, bbox_inches=None, pad_inches=0.1)
    plt.close()

<a class = "Anchor" id = "IRAF_Compatible"> </a>
# Hacer compatible con IRAF

Los productos terminados de este notebook se ajustan a los estándares de los Dragons Fits que no cumplen con lo que IRAF espera. Si desea que los espectros reducidos finales sean compatibles con IRAF, puede usar la receta MakeIRAFCompatible como se muestra a continuación. (Descoplamiento antes de correr).

Ejecutar la primera celda creará un archivo llamado S20230416S0079_RED001_Dragons_IRAFCompatible.fits.

Ejecutar la segunda celda creará un archivo llamado S20230416S0079_BLUE001_Dragons_IRAFCompatible.fits.

In [ ]:
# REDET_IRAF = REDUCT ()
# RED_DRAGONS_FILES = ['S20230416S0079_RED001_DRAGONS.FITS']
# REDW_IRAF.FILES.EXTEND (RED_DRAGONS_FILES)
# REDW_IRAF.RECIPENAME = 'MakeIRAFCompatible'
# REDW_IRAF.RUNR ()

In [ ]:
# REDET_IRAF = REDUCT ()
# blue_dragons_files = ['S20230416S0079_blue001_dragons.fits']
# REDW_IRAF.FILES.EXTEND (Blue_dragons_files)
# REDW_IRAF.RECIPENAME = 'MakeIRAFCompatible'
# REDW_IRAF.RUNR ()

Este notebook solo ha usado las opciones predeterminadas de los dragones. Si desea que todas las exposiciones individuales se reduzcan por separado, puede investigar <a href = "https://dragons.readthedocs.io/projects/ghost-drtutorial/en/stable/ex1_ghost_stdonetarget_cmdline.html#ternative-data-products"> Combineorders () </s) </aa>.

## Opcional: limpiar el directorio de trabajo. (descomentar antes de ejecutar)

In [ ]:
# clean_up (save_reduced = 0)